In [1]:
import requests
import os
import json
from api_keys import BEARER_TOKEN
from tqdm import tqdm
import os
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Funções (Análise Exploratória)

### Função para plotar bar plot com a contagem de tokens

In [2]:
def plot_bar_count_words(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = CountVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para plotar bar plot com tf-idf

In [3]:
def plot_bar_tf_idf(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = TfidfVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    "MAX":data_vect.std(axis=0)
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para contagem de tokens

In [4]:
def calculate_number_words(text):

    quantity_of_words = text.split(" ")

    quantity_of_words = [i for i in quantity_of_words if i!=""]

    quantity_of_words = len(quantity_of_words)

    return quantity_of_words


### Função para contagem de diferentes tokens

In [5]:
def calculate_number_diferent_words(text):

    quantity_of_diferent_words = text.split(" ")

    quantity_of_diferent_words = [i for i in quantity_of_diferent_words if i!=""]

    quantity_of_diferent_words = set(quantity_of_diferent_words)

    quantity_of_diferent_words = list(quantity_of_diferent_words)

    quantity_of_diferent_words = len(quantity_of_diferent_words)

    return quantity_of_diferent_words


### Função para criar textos sem repetição de palavras para ser utilizado na análise exploratória 

In [6]:
def convert_text_to_no_repeat_words(text):

    text_with_no_repeat_words = text.split(" ")

    text_with_no_repeat_words = [i for i in text_with_no_repeat_words if i!=""]

    text_with_no_repeat_words = set(text_with_no_repeat_words)

    text_with_no_repeat_words = list(text_with_no_repeat_words)

    text_with_no_repeat_words = " ".join(text_with_no_repeat_words)

    return text_with_no_repeat_words

### Função para o pré-processamento do texto 

In [7]:
def text_cleaner(text):
    
    nltk_stopwords = stopwords.words('portuguese')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

# Funções (Extração de Tweets)

In [8]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(query = "@globoplay -is:retweet",until_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&until_id={}".format(
            query,until_id
        )
        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100".format(
            query
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = "@BBB -is:retweet",until_id=None):
    bearer_token = BEARER_TOKEN
    url = create_url(query,until_id)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    return json_response

def extract_many_tweets(qnt_cycle=10,folder="data_tweets",start_from_id=None,query="@BBB"):
    
    
    oldest_id = None
    
    for i in tqdm(range(qnt_cycle)):
    
        
        if i == 0:
            
            #extract the 100 tweets first
            
            if start_from_id:
        
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=None)
            
            else:
                
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=start_from_id)
                
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            #get the current date
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            oldest_date = date_extraction
            
            df_data_tweets = df_data_tweets_temp.copy()
            
            # name file
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file,sep=",")
            
    
            
        else:
            
            
            #extract more 100 tweets older
            
            data_tweets_temp = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=oldest_id)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets_temp["data"])
            
            
            #get the current date
            
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            oldest_id = data_tweets_temp['meta']['oldest_id']
            
            df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])
            
            date_extraction = datetime.now()
            
            df_data_tweets.reset_index(inplace=True,drop=True)
            
            
            # remove old files
            
            os.remove(name_file)
            
            
            # name file
            
            oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file.format(folder),sep=",")
            
            

    return df_data_tweets
    
    


# Extração de Tweets

In [89]:
data_tweets_final = extract_many_tweets(qnt_cycle=10,folder="data_tweets",query="elonmusk")#,start_from_id="1367600965277384706")

100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


In [90]:
data_tweets_final.head(30)

,id,text,date_extraction
0,1383642437072027649,@DanielRegha @elonmusk Wouldn’t you agree if y...,2021-04-18 01:44:09.586137
1,1383642433838219264,@ProtocolSigma @HuobiGlobal @binance @ethereum...,2021-04-18 01:44:09.586137
2,1383642433745948684,@elonmusk Tweets are publicly visible by defau...,2021-04-18 01:44:09.586137
3,1383642431778824194,@elonmusk News! i have this new crpto sh0w. \n...,2021-04-18 01:44:09.586137
4,1383642411583238151,@LAPEACEFUL @SHIBA35022474 @Coin_Shark @Shibto...,2021-04-18 01:44:09.586137
5,1383642397637103621,"@elonmusk In publishing and graphic design, Lo...",2021-04-18 01:44:09.586137
6,1383642387650519040,@JosieLong @elonmusk That's literally what the...,2021-04-18 01:44:09.586137
7,1383642386476109828,Two days after many many traders and investors...,2021-04-18 01:44:09.586137
8,1383642381921112064,@elonmusk A PATTERN RECOGNITION OF THE HIGHEST...,2021-04-18 01:44:09.586137
9,1383642377521229825,@GusTheInvestor @LarckeningXuruo @elonmusk Fal...,2021-04-18 01:44:09.586137


In [63]:
def function_to_calc_histogram(x,initial_interval, final_interval,n_bins,indice=False):

    interval = np.linspace(initial_interval, final_interval, num=n_bins)

    for j,i in enumerate(interval):


        if i == interval[len(interval)-1]:

            if x>=i:

                
                if indice:

                    return j
                
                else:
                    
                    return "{}<".format(x)


        else:

            if x>=i and x<interval[j+1]:

                inicial = round(i, 1)

                final = round(interval[j+1],1)

            
                if indice:

                    return j
                
                else:
                    
                    return "[{},{})".format(inicial,final)


# Criação de uma coluna com os textos sem repetição de palavras para ser utilizado na análise exploratória

In [64]:
data_tweets_final['text_unique_words'] = data_tweets_final['text'].apply(lambda x: convert_text_to_no_repeat_words(x))

# Calculo Número de tokens

In [94]:
data_tweets_final['number_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_words(x))

# Calculo Número de diferentes tokens

In [66]:
data_tweets_final['number_diferent_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_diferent_words(x))

# Máximo número de tokens

In [67]:
max_count = data_tweets_final["number_tokens"].max()

In [95]:
data_tweets_final["number_tokens"].var()

187.4528505194297

# Mínimo número de tokens

In [68]:
min_count = data_tweets_final["number_tokens"].min()

## Dados para o histograma 

In [69]:
data_tweets_final['bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=False))

58<


In [70]:
data_tweets_final['indices_bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=True))

58<


In [73]:
data_histogram = data_tweets_final.groupby(["bins","indices_bins"]).sum().sort_values(by=["indices_bins"])

In [75]:
data_histogram.reset_index(drop=False,inplace=True)

In [78]:
data_histogram

,bins,indices_bins,number_tokens,number_diferent_tokens
0,"[1.0,7.3)",0,630,629
1,"[7.3,13.7)",1,2271,2232
2,"[13.7,20.0)",2,2931,2819
3,"[20.0,26.3)",3,2492,2357
4,"[26.3,32.7)",4,2329,2183
5,"[32.7,39.0)",5,2643,2381
6,"[39.0,45.3)",6,3954,3485
7,"[45.3,51.7)",7,1893,1638
8,"[51.7,58.0)",8,2074,1851
9,58<,9,58,52


In [77]:
data_histogram["bins"].tolist()

['[1.0,7.3)',
 '[7.3,13.7)',
 '[13.7,20.0)',
 '[20.0,26.3)',
 '[26.3,32.7)',
 '[32.7,39.0)',
 '[39.0,45.3)',
 '[45.3,51.7)',
 '[51.7,58.0)',
 '58<']

In [79]:
data_histogram["number_tokens"].tolist()

[630, 2271, 2931, 2492, 2329, 2643, 3954, 1893, 2074, 58]

# DF top 10 MEAN

In [53]:
df_report_mean = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [54]:
df_report_mean["MEAN"].tolist()

[1.02, 0.348, 0.299, 0.299, 0.298, 0.264, 0.246, 0.238, 0.194, 0.194]

In [55]:
df_report_mean["WORDS"].tolist()

['boninho',
 'bbb21',
 'https',
 'co',
 'que',
 'juliette',
 'de',
 'bbb',
 'camilla',
 'fiuk']

# DF top 10 SUM docs

In [80]:
df_report_sum = plot_bar_count_words(text_column='text_unique_words',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [81]:
df_report_sum["P_DOCS"] =  df_report_sum["SUM"]/len(data_tweets_final)

In [82]:
df_report_sum

,SUM,WORDS,P_DOCS
2240,799,fgv,0.804632
1139,704,co,0.708963
2664,704,https,0.708963
1479,490,de,0.493454
1445,389,da,0.391742
1858,318,em,0.320242
4481,233,que,0.234642
1720,197,do,0.198389
3701,152,na,0.153072
1167,130,com,0.130916


# DF top 10 SUM

In [ ]:
df_report_sum_docs = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [ ]:
df_report_sum_docs

# DF top 10 MEAN TF-IDF

In [ ]:
df_report_tfidf_mean = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [ ]:
df_report_tfidf_mean

# DF top 10 MAX TF-IDF

In [ ]:
df_report_tfidf_max = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MAX',top=10,return_df=True)

In [ ]:
df_report_tfidf_max